In [1]:
using DataFrames
using GLM

In [33]:
train=readtable("./train.csv")
store=readtable("./store.csv")
test=readtable("./test.csv")

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
1,1,1,4,2015-09-17,1,1,0,0
2,2,3,4,2015-09-17,1,1,0,0
3,3,7,4,2015-09-17,1,1,0,0
4,4,8,4,2015-09-17,1,1,0,0
5,5,9,4,2015-09-17,1,1,0,0
6,6,10,4,2015-09-17,1,1,0,0
7,7,11,4,2015-09-17,1,1,0,0
8,8,12,4,2015-09-17,1,1,0,0
9,9,13,4,2015-09-17,1,1,0,0
10,10,14,4,2015-09-17,1,1,0,0


In [ ]:
function parseDates(dateCol)
    dates = DataArray(Date, length(dateCol))
    for i=1:size(dateCol,1)
        dates[i] = Date(dateCol[i])
    end
    return dates
end
# train[:Date] = parseDates(train[:Date])
# test[:Date] = parseDates(test[:Date])

In [ ]:
function center(df, colName)
    return df[colName] .- mean(train[colName])
end
# train[:Customers] = center(train, :Customers)
# test[:Customers] = center(test, :Customers)

In [34]:
function stringToBool(holidayCol)
    data = DataArray(Bool, length(holidayCol))
    for i=1:size(holidayCol,1)
        data[i] = ("0" != holidayCol[i])
    end
    return data
end
train[:StateHoliday] = stringToBool(train[:StateHoliday])
test[:StateHoliday] = stringToBool(test[:StateHoliday])
train[:SchoolHoliday] = convert(DataArray{Bool,1}, train[:SchoolHoliday])
test[:SchoolHoliday] = convert(DataArray{Bool,1}, test[:SchoolHoliday])
train[:Open] = convert(DataArray{Bool,1}, train[:Open])
test[:Open] = convert(DataArray{Bool,1}, test[:Open])

41088-element DataArrays.DataArray{Bool,1}:
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
    ⋮
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true

# Poisson Regression

In [50]:
fitByStoreOpen = by(
    train[train[:Open] .== true, :],
    :Store,
    df -> glm(Sales~DayOfWeek+Promo+SchoolHoliday, df, Poisson())
)
fitByStore = by(
    train,
    :Store,
    df -> glm(Sales~DayOfWeek+Promo+SchoolHoliday, df, Poisson())
)

,Store,x1
1,1,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.5243 0.00144454 5901.06 <1e-99 DayOfWeek -0.101863 0.000289065 -352.388 <1e-99 Promo 0.324801 0.00111072 292.424 <1e-99 SchoolHoliday -0.0301474 0.00128329 -23.4922 <1e-99"
2,2,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.66301 0.00142385 6084.22 <1e-99 DayOfWeek -0.166902 0.000294243 -567.226 <1e-99 Promo 0.534118 0.00108408 492.694 <1e-99 SchoolHoliday 0.0169712 0.00125788 13.4919 <1e-40"
3,3,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.90882 0.00124402 7161.3 <1e-99 DayOfWeek -0.146221 0.000250912 -582.756 <1e-99 Promo 0.563415 0.00092372 609.941 <1e-99 SchoolHoliday 0.0261214 0.00106745 24.471 <1e-99"
4,4,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 9.23718 0.00102218 9036.78 <1e-99 DayOfWeek -0.103984 0.000204257 -509.082 <1e-99 Promo 0.297838 0.00077771 382.968 <1e-99 SchoolHoliday 0.053277 0.000900343 59.1742 <1e-99"
5,5,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.61377 0.00149083 5777.85 <1e-99 DayOfWeek -0.182246 0.000307329 -593.001 <1e-99 Promo 0.581076 0.00112546 516.299 <1e-99 SchoolHoliday 0.0473819 0.00129607 36.5581 <1e-99"
6,6,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.78947 0.00136524 6438.05 <1e-99 DayOfWeek -0.154707 0.000277137 -558.232 <1e-99 Promo 0.434667 0.00103096 421.613 <1e-99 SchoolHoliday -0.02722 0.00122516 -22.2174 <1e-99"
7,7,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 9.22731 0.00105877 8715.14 <1e-99 DayOfWeek -0.148262 0.000217498 -681.671 <1e-99 Promo 0.412413 0.000806588 511.305 <1e-99 SchoolHoliday 0.124444 0.000936357 132.902 <1e-99"
8,8,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.78522 0.00134394 6536.91 <1e-99 DayOfWeek -0.17191 0.00027885 -616.496 <1e-99 Promo 0.553685 0.00102655 539.364 <1e-99 SchoolHoliday -0.0117816 0.00121927 -9.66289 <1e-21"
9,9,"DataFrames.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Poisson,GLM.LogLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Float64}: Coefficients: Estimate Std.Error z value Pr(>|z|) (Intercept) 8.90052 0.00126214 7051.91 <1e-99 DayOfWeek -0.127063 0.000252831 -502.562 <1e-99 Promo

In [55]:
function makePreds(df)
    preds = DataArray(Float64, size(df,1))
    preds[!isna(df[:Open]) & (df[:Open] .== false)] = 0
    for i=unique(df[:Store])
        modelKnownOpen = fitByStoreOpen[fitByStoreOpen[:Store] .== i, :x1][1]
        modelUnknownOpen = fitByStore[fitByStore[:Store] .== i, :x1][1]
        mask = !isna(df[:Open]) & (df[:Open] .== true) & (df[:Store] .== i)
        preds[mask, :] = predict(modelKnownOpen, df[mask, :])
        mask = isna(df[:Open]) & (df[:Store] .== i)
        preds[mask, :] = predict(modelUnknownOpen, df[mask, :])
    end
    return preds
end

makePreds (generic function with 1 method)

In [25]:
trainPreds = makePreds(train)
mean(abs(makePreds(train) - preds))

1017209-element DataArrays.DataArray{Float64,1}:
  4911.85
  5343.52
  8161.21
 11540.4 
  4892.53
  5221.16
 14560.4 
  7452.21
  8328.02
  6833.47
  9351.75
  8700.76
  8697.13
     ⋮   
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 
     0.0 

In [56]:
testPreds = makePreds(test)

41088-element DataArrays.DataArray{Float64,1}:
  5361.01
  8314.23
  9626.39
  6462.1 
  7398.76
  6137.12
  8823.63
  9013.31
  6453.21
  6243.85
  7778.1 
  8612.29
  7513.95
     ⋮   
  6140.77
  4291.94
  5180.63
  3588.29
  3436.56
  4673.59
  3591.42
  3531.96
  7645.53
  5803.06
 20133.0 
  5807.61

In [57]:
writetable("output.csv", DataFrame(Id=1:length(testPreds), Sales=testPreds))

# Gaussian Process

In [58]:
Pkg.add("GaussianProcesses")


INFO: Cloning cache of GaussianProcesses from git://github.com/STOR-i/GaussianProcesses.jl.git
INFO: Installing GaussianProcesses v0.1.4
INFO: Package database updated


In [59]:
using GaussianProcesses

In [ ]:
?